# 정부혁신1번가 - 정부혁신 홈페이지
* 정부혁신 제안(목록) > 정부혁신국민포럼 

출처 : https://www.innogov.go.kr/ucms/ogp/sug/list.do?menuNo=300011

```
ㆍ다함께 잘 사는 나라, 늘 혁신하는 정부를 위한 의견이 있다면 자유롭게 제안해 주세요.
ㆍ내가 낸 정부혁신제안이 추천 30표를 받으면 토론이 진행됩니다.
ㆍ추천 30표를 받지 못한 경우에도 정부혁신국민포럼 운영위원단의 논의를 통해 중요성과 시급성이 높다면 토론으로 바로 이관될 수 있습니다.
ㆍ타인의 명예를 훼손하는 내용 혹은 허위사실, 특정집단에 대한 혐오 표현 등 부적절한 제안은 관리자에 의해 숨김 처리 혹은 삭제될 수 있습니다.
```

## 크롤링 전 확인 하기
* https://www.innogov.go.kr/robots.txt
* 저작권 확인하기

In [1]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
import pandas as pd
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러올 수 있습니다.
import requests
# request로 가져온 웹사이트의 html 태그를 파싱하는데 사용합니다.
from bs4 import BeautifulSoup as bs
# 랜덤숫자를 생성한다.
import random
import time
# 대량 데이터 처리시 진행 상황을 표시합니다.
from tqdm import tqdm, trange
# 정규표현식
import re

## 정부혁신 국민포럼 페이지 가져오기
* 전체 내용을 가져오기 위해 목록에 있는 href 태그 안에 있는 url을 가져옵니다.

In [2]:
#제안시작월 선택
#basei url: https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum=1&menuNo=300011&cateCd=&status1Cd=&Status2Cd=&searchText=&sugMonthTp=202006&orderKey=registDtDesc
pnum = 1
year_month = 201906

base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&sugMonthTp={year_month}&orderKey=registDtDesc"
base_url

'https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum=1&menuNo=300011&sugMonthTp=201906&orderKey=registDtDesc'

In [3]:
response = requests.get( base_url )
response

if response.status_code == 200:
    html = bs(response.text, 'html.parser')
    # 소스코드를 제대로 가져왔는지 확인해 봅니다.
    tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
    print(tags)
    for tag in tags:
        print(tag['href']) #url만 가져옴(첫번째 페이지의 목록만 가져옴)

[<a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=217&amp;pnum=1">근린공원</a>, <a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=216&amp;pnum=1">공시송달 의뢰공문의 전국배포를 중단하여 문서로 인한 스...</a>, <a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=215&amp;pnum=1">중앙부처 및 산하기관에서 추진하는 공모사업, 지원 사업...</a>, <a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=214&amp;pnum=1">연가 기부제 도입</a>, <a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=210&amp;pnum=1">어린이집교사 근무환경개선</a>]
/ucms/ogp/sug/view.do?menuNo=300011&sgId=217&pnum=1
/ucms/ogp/sug/view.do?menuNo=300011&sgId=216&pnum=1
/ucms/ogp/sug/view.do?menuNo=300011&sgId=215&pnum=1
/ucms/ogp/sug/view.do?menuNo=300011&sgId=214&pnum=1
/ucms/ogp/sug/view.do?menuNo=300011&sgId=210&pnum=1


### URL이 있는 태그의 위치
* 브라우저의 Inspect 기능을 통해 링크에 해당되는 selector를 가져오기
* Copy -> Copy selector를 통해 해당 태그의 위치를 가져옴

In [4]:
#content > div.suggestion_list > ul > li > dl > dt > p > a : copy selcector


### 함수로 구현하기 전에 미리 원하는 정보를 가져와서 출력해 봄

In [5]:
response = requests.get( base_url )
response

if response.status_code == 200:
    html = bs(response.text, 'html.parser')
    # 소스코드를 제대로 가져왔는지 확인해 봅니다.
    tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a') 
    print(tags) # 각 페이지의 제목을 가져올수 있다.

[<a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=217&amp;pnum=1">근린공원</a>, <a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=216&amp;pnum=1">공시송달 의뢰공문의 전국배포를 중단하여 문서로 인한 스...</a>, <a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=215&amp;pnum=1">중앙부처 및 산하기관에서 추진하는 공모사업, 지원 사업...</a>, <a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=214&amp;pnum=1">연가 기부제 도입</a>, <a href="/ucms/ogp/sug/view.do?menuNo=300011&amp;sgId=210&amp;pnum=1">어린이집교사 근무환경개선</a>]


### 위에서 실습한 내용을 페이지 번호를 추가해 가면서 크롤링해 오는 함수를 작성

In [6]:
# 연-월별 전체 제안 목록을 가져옵니다.
def get_suggestion_list(pnum):
    # print(f'year_month : {year_month}, pnum: {pnum}')
    base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&orderKey=registDtDesc"
    # base_url 페이지를 받아온다.
    response = requests.get( base_url )
    # status_code 가 200 OK 일 때 아래의 코드를 실행한다.
    if response.status_code == 200:
        # 웹페이지의 내용을 가져왔다면 뷰티풀숩을 통해 html 태그를 가져온다.
        html = bs(response.text, 'html.parser')
        # html 태그 내용 중 아래의 selector를 통해 목록에 있는 다른 게시글을 가져오도록 한다.
        tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
        # tags 의 내용이 없다면 suggestion_list 를 반환한다.
        if not tags :
            return(suggestion_list)
        else :
            # tags 에서 다른 게시글을 가져와 suggestion_list 에 담아준다.
            for tag in tags:
                suggestion_list.append(tag['href']) #sugesstion list에 링크를 추가
        pnum += 1
        # 페이지번호를 하나씩 더해서 재귀호출을 합니다.
        get_suggestion_list(pnum)
        print(pnum)
    else:
        return(suggestion_list)

In [7]:
# 목록의 내용만 크롤링 해옵니다.
# 목록의 링크 값을 가져와 suggestion_list 리스트에 담아줍니다.
suggestion_list = []
pnum = 1

get_suggestion_list(pnum)
suggestion_list    

49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2


['/ucms/ogp/sug/view.do?menuNo=300011&sgId=280&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=279&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=278&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=277&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=276&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=275&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=274&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=272&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=271&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=270&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=269&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=268&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=267&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=266&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=265&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=264&pnum=4',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=263&pnum=4',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=262&p

### 제대로 href 태그의 링크를 가져왔는지 확인하기 

In [8]:
print(len(suggestion_list))

suggestion_list[:3]

236


['/ucms/ogp/sug/view.do?menuNo=300011&sgId=280&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=279&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=278&pnum=1']

## 게시물 내용 가져오기
* href 목록을 통해 게시물의 내용을 가져오기

In [9]:
url = '/ucms/ogp/sug/view.do?menuNo=300011&sgId=278&pnum=1'
base_url = f"https://www.innogov.go.kr{url}"
response = requests.get( base_url )
if response.status_code == 200: #response가 정상이면 동작
    html = bs(response.text, 'html.parser')
    #content > div.vveiw_box1 > dl > dt
    title = html.select(
            '#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
    print(title)

원스톱 의원요구자료 제출 시스템 구축


In [71]:
desc = html.select(
            '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd') 
category = desc[0].get_text(strip=True)
print(desc)
print(desc[0].get_text(strip=True))
print(desc[1].get_text(strip=True))
print(desc[2].get_text(strip=True))
print(desc[3].get_text(strip=True))

[<dd>일반행정</dd>, <dd>2019.10.15</dd>, <dd>2020.01.12</dd>, <dd>연시랑</dd>]
일반행정
2019.10.15
2020.01.12
연시랑


In [72]:
# id값을 저장해 주기 위해 미리 추출 해보기
u = 'view.do?menuNo=300011&sgId=150&pnum=1'
u.split('=')[2].split('&')[0]

'150'

In [73]:
# 목록 리스트에 있는 url를 넘겨주면 내용을 크롤링 해옵니다.
def get_suggestion_content(url):
    article = []
    base_url = f"https://www.innogov.go.kr/{url}"
    response = requests.get( base_url )
    if response.status_code == 200:
        html = bs(response.text, 'html.parser')
        # 제목을 가져옵니다.
        title = html.select(
            '#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
        # dd 태그에 있는 내용을 가져와서 카테고리, 시작일, 마감일, 제안인 정보를 추출합니다.
        desc = html.select(
            '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
        category = desc[0].get_text(strip=True)
        content = html.select(
            '#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)
        start = desc[1].get_text(strip=True)
        end = desc[2].get_text(strip=True)
        author = desc[3].get_text(strip=True)
        # 추천인원을 가져옵니다.
        vote = html.select('#counter')[0].get_text(strip=True)
        # 해당 게시물의 Id 값을 가져옵니다.
        sgId = url.split('=')[2].split('&')[0]
        
        article.append(sgId)
        article.append(title)
        article.append(category)
        article.append(content)
        article.append(start)
        article.append(end)
        article.append(vote)
        article.append(author)
       # print(article)
        
#       time.sleep(random.randint(1,2))
        return article


### 각 게시물의 세부내용 가져오기

In [74]:
data = []
# tqdm 을 사용해서 어느정도 가져왔는지 확인하기
for i, suggestion in tqdm(enumerate(suggestion_list)): # enumerate를 통해 i(index), suggestion을 가져옴
    # suggestion_list 에 있는 각 게시물에 접근해서 카테고리, 시작일, 마감일, 제안인, 제목, 내용 등을 가져옵니다.
    s = get_suggestion_content(suggestion)
    data.append(s)

236it [00:43,  5.42it/s]


In [75]:
column_names = ["sgId", "title", "category", "content", "start", "end", "vote", "author"]
column_names

['sgId', 'title', 'category', 'content', 'start', 'end', 'vote', 'author']

In [76]:
#data

In [77]:
data = pd.DataFrame(data, columns=column_names)
data.head()

,sgId,title,category,content,start,end,vote,author
0,280,행정기관 소속 위원회 위원 개인정보 및 안건 공개 기준에 관한 법적 근거 마련,일반행정,"ㅁ 현황ㅇ 법령에 따라 설치되는 행정기관 소속 위원회들이 다수 운영되고 있으며, ...",2019.10.16,2020.01.13,0,법제혁신
1,279,초과근무 줄이기!,일반행정,1. 문제점* 초과근무를 많이 할수록 일을 열심히 하고 있다고 생각하는 조직 문화*...,2019.10.16,2020.01.13,2,ㅁㄴㅁ
2,278,원스톱 의원요구자료 제출 시스템 구축,일반행정,​​□ 제안제목○ 원스톱 의원요구자료 제출 시스템 구축□현황 및 문제점○ 각부처 별...,2019.10.15,2020.01.12,0,연시랑
3,277,예산을 공평하게 분배,사회통합,제목 : 예산을 부처별 공평하게 분배후 차후평가문제점 : 1. 예산편성 삭감원칙으...,2019.10.15,2020.01.12,3,슈퍼맨119
4,276,공무원 인트라넷 통합메신저,일반행정,"부처별,기관별로 자체 인트라넷 메신저가 활성화되며,신속한 업무 추진과 협조에 큰 도...",2019.10.15,2020.01.12,0,페이소스


In [63]:
data.tail()

,sgId,title,category,content,start,end,vote,author
231,5,철도 및 고속도로변 유휴지에 태양전지판 설치,기타,에너지 부존자원 국가인 우리나라에서 국토를 좀 더 효율적으로 활용하여 미래의 무공해...,2018.08.03,2018.09.02,8,지고기
232,4,“일거리” 매칭 사이트를 개설하여 “일자리” 창출 연계,상생협력,○문제점-최근(2018.05)취업자 증가폭이10만명 밑으로 추락했다고 한다.글로벌 ...,2018.08.02,2018.09.01,4,옹성산
233,3,경력증명을 위한 경력등록(은행)제도 시행,일반행정,"○문제점-모든 기관(국가,민간)에 사업,채용지원,이력증빙서류로“경력증명서”를 요구하...",2018.08.02,2018.09.01,4,옹성산
234,2,공무원 전입시험의 투명성 확보방안,반부패청렴,현직 공무원들의 타기관이나 상급기관에 전입시험시투명성을 높이는 방안을 마련했으면 합...,2018.08.01,2018.08.31,7,음지가양지된다
235,1,어린이집 차량내 어린이 질식사고 등에 대한 상세한 대책이 필요하다.,안전,"최근 동두천 어린이집 차량내 어린이 질식사고가 발생하여, 이에 대한 여러가지 대책을...",2018.07.24,2018.08.23,14,헌비


In [80]:
data["category"].value_counts()

일반행정     41
안전       33
보건복지     32
기타       31
인권/평등    21
국민참여     17
상생협력     15
환경       14
사회통합     13
공동체      10
반부패청렴     9
Name: category, dtype: int64

In [65]:
data.to_csv("suggestion.csv", index=False)

In [66]:
pd.read_csv("suggestion.csv").head()

,sgId,title,category,content,start,end,vote,author
0,280,행정기관 소속 위원회 위원 개인정보 및 안건 공개 기준에 관한 법적 근거 마련,일반행정,"ㅁ 현황ㅇ 법령에 따라 설치되는 행정기관 소속 위원회들이 다수 운영되고 있으며, ...",2019.10.16,2020.01.13,0,법제혁신
1,279,초과근무 줄이기!,일반행정,1. 문제점* 초과근무를 많이 할수록 일을 열심히 하고 있다고 생각하는 조직 문화*...,2019.10.16,2020.01.13,2,ㅁㄴㅁ
2,278,원스톱 의원요구자료 제출 시스템 구축,일반행정,​​□ 제안제목○ 원스톱 의원요구자료 제출 시스템 구축□현황 및 문제점○ 각부처 별...,2019.10.15,2020.01.12,0,연시랑
3,277,예산을 공평하게 분배,사회통합,제목 : 예산을 부처별 공평하게 분배후 차후평가문제점 : 1. 예산편성 삭감원칙으...,2019.10.15,2020.01.12,3,슈퍼맨119
4,276,공무원 인트라넷 통합메신저,일반행정,"부처별,기관별로 자체 인트라넷 메신저가 활성화되며,신속한 업무 추진과 협조에 큰 도...",2019.10.15,2020.01.12,0,페이소스


In [67]:
data.shape

(236, 8)